In [2]:
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

In [3]:
loader = PyPDFDirectoryLoader("./us_census")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

final_documents = text_splitter.split_documents(documents)
final_documents[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.2 (Windows)', 'creationdate': '2023-09-09T07:52:17-04:00', 'author': 'U.S. Census Bureau', 'keywords': 'acsbr-015', 'moddate': '2023-09-12T14:44:47+01:00', 'title': 'Health Insurance Coverage Status and Type by Geography: 2021 and 2022', 'trapped': '/false', 'source': 'us_census/acsbr-015.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, w

In [4]:
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5", #or sentence-transformers/all-MiniLM-16-v2
    model_kwargs = {'device':'cpu'},
    encode_kwargs = {'normalize_embeddings':True}
)

/tmp/ipykernel_140385/901316050.py:1: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  huggingface_embeddings = HuggingFaceBgeEmbeddings(
/home/ppk/Documents/LLM_Project/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import numpy as np
np.array(huggingface_embeddings.embed_query(final_documents[0].page_content))

array([-0.07903482, -0.01134111, -0.02312097,  0.02844462,  0.05053343,
        0.05317824, -0.01907787,  0.03456026, -0.1021137 , -0.02915704,
        0.08524264,  0.05650728, -0.02545437, -0.03308489, -0.00635733,
        0.04090864, -0.00628108,  0.00356743, -0.03854128,  0.03667685,
       -0.04289804,  0.03425251, -0.031169  , -0.03793724,  0.01728393,
        0.01214925,  0.00653121,  0.01463565, -0.05529054, -0.15320709,
        0.00730846,  0.03202943, -0.04701128, -0.01595973,  0.01874445,
        0.02642935, -0.02306378,  0.08438038,  0.04182486,  0.05278175,
       -0.03057603,  0.01564258, -0.01689074,  0.00529412, -0.02417434,
        0.00412998, -0.01889937, -0.00150624, -0.0083694 , -0.03390064,
        0.03515957, -0.00553132,  0.04910938,  0.05971858,  0.05615962,
       -0.05105155,  0.01475132, -0.01849959, -0.03284642,  0.03576626,
        0.04947704, -0.0093888 , -0.26202121,  0.09750332,  0.0171569 ,
        0.04781386, -0.0055632 , -0.00298308, -0.02207357, -0.04

In [6]:
vectorstore = FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [7]:
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_documents = vectorstore.similarity_search(query)

print(relevant_documents[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [8]:
retriever = vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x728f89f8f610> search_kwargs={'k': 3}


In [9]:
import os
from dotenv import load_dotenv

load_dotenv()

hf_api_key = os.environ['HUGGINGFACE_TOKEN']


In [10]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id = "mistralai/Mistral-7B-v0.1",
    task="text-generation",
 #   huggingfacehub_api_token = hf_api_key,
    pipeline_kwargs = {"temperature":0.1, "max_new_tokens":500}
)
query = "What is the health insurance coverage?"
hf.invoke(query)

Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.54s/it]
Device set to use cpu
/home/ppk/Documents/LLM_Project/env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


'What is the health insurance coverage?\n\nThe health insurance coverage is a contract between the insurer and the insured. The insurer agrees to pay the insured for the medical expenses incurred by the insured. The insured agrees to pay the premiums to the insurer.\n\nWhat is the health insurance coverage?\n\nThe health insurance coverage is a contract between the insurer and the insured. The insurer agrees to pay the insured for the medical expenses incurred by the insured. The insured agrees to pay the premiums to the insurer.\n\nWhat is the health insurance coverage?\n\nThe health insurance coverage is a contract between the insurer and the insured. The insurer agrees to pay the insured for the medical expenses incurred by the insured. The insured agrees to pay the premiums to the insurer.\n\nWhat is the health insurance coverage?\n\nThe health insurance coverage is a contract between the insurer and the insured. The insurer agrees to pay the insured for the medical expenses incurr

In [11]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

prompt = PromptTemplate(template=prompt_template, input_variables=["context","question"])

In [12]:
retrievalQA = RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [13]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [14]:
result = retrievalQA.invoke({'query':query})
print(result['result'])

/home/ppk/Documents/LLM_Project/env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

comparison of ACS and CPS ASEC measures 
of health insurance coverage, refer to <www.
census.gov/topics/health/health-insurance/
guidance.html>.
9 Respondents may have more than one 
health insurance coverage type at the time 
of interview. As a result, adding the total 
number of people with private coverage and 
the total number with public coverage will 
sum to more than the total number with any 
coverage.
• From 2021 to 2022, nine states 
reported increases in private 
coverage, while seven reported 
decreases (Appendix Table B-2). 
DIFFERENCES IN THE 
UNINSURED RATE BY STATE 
IN 2022
In 2022, uninsured rates at the 
time of interview ranged across 
states from a low of 2.4 percent 
in Massachusetts to a high of 16.6 
percent in Texas, compared to the 
national rate of 8.0 percent.10 Ten 
of the 15 states with uninsured 
10 The uninsured rates in the Distri